# BIP0039 生成确定性密钥的助记码

<https://en.bitcoin.it/wiki/BIP_0039>

**BTC私钥** 实际上就是一个32字节的随机数组，而 **公钥** 是通过将椭圆曲线的 **生成元点** 自加而得到，自加的次数就是这个私钥 ，此时私钥被当作一个大整数。而 **BTC地址** 则是通过把公钥哈希而得到。程序逻辑如下：

In [13]:
import random

# 固定随机种子，免得每次跑的结果不一样
random.seed(10)

# 随机生成一个私钥
private_key = random.randint(0, 2**256 - 1)
print(f"private_key: 0x{private_key:064x}")

private_key: 0x854a965708ceac392904cdefcf84b683a749f9c5470b9805d2d6b8777dc59a3a


In [20]:
# BTC所选择的椭圆曲线是secp256k1, 其参数如下
p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
a = 0
b = 7
n = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141
Gx = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798
Gy = 0x483ADA7726A3C4655DA4FBFC0E1108A8FD17B448A68554199C47D08FFB10D4B8

# 椭圆曲线的方程是 y^2 = x^3 + ax + b

# 椭圆曲线上的加法如下，其中 None 表示无穷远点，即加法的单位元，即 0，p1和p2是椭圆曲线上的点
def secp256k1_add(p1, p2):
    if p1 is None:
        return p2
    if p2 is None:
        return p1
    if p1[0] == p2[0] and p1[1] != p2[1]:
        # 两个点互为相反数，即相加为0
        assert (p1[1] + p2[1] % p) == 0
        return None
    # s是斜率
    if p1[0] == p2[0]:
        s = (3 * p1[0] * p1[0] + a) * pow(2 * p1[1], -1, p)
    else:
        s = (p2[1] - p1[1]) * pow(p2[0] - p1[0], -1, p)
    x = (s * s - p1[0] - p2[0]) % p
    y = ( - (s * (x - p1[0]) + p1[1]) ) % p
    return (x, y)

# 私钥 -> 公钥 (倍点算法)
def priv_key_to_pub_key(priv_key):
    # 生成公钥
    pub_key = None
    for i in range(256):
        if priv_key & (1 << i):
            if pub_key is None:
                pub_key = (Gx, Gy)
            else:
                pub_key = secp256k1_add(pub_key, (Gx, Gy))
    return pub_key

# 生成公钥
pub_key = priv_key_to_pub_key(private_key)
print(f"pub_key: 0x{pub_key[0]:064x}, 0x{pub_key[1]:064x}")

pub_key: 0x34ff3be4033f7a06696c3d09f7d1671cbcf55cd700535655647077456769a24e, 0x5d9d11623a236c553f6619d89832098c55df16c3e8f8b6818491067a73cc2f1a
